<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/MYEMB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## MNIST Digit Recognition

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

# Load MNIST dataset
train_dataset = datasets.MNIST(
    root="./data", train=True, download=True, transform=transforms.ToTensor()
)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)

# Load MNIST test dataset
test_dataset = datasets.MNIST(
    root="./data", train=False, download=True, transform=transforms.ToTensor()
)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

# Define the ImageEBM with mixed activations
class ImageEBM(nn.Module):
    def __init__(self):
        super(ImageEBM, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(8)
        self.fc1 = nn.Linear(8 * 28 * 28, 10)
        self.fc2 = nn.Linear(10, 1)

    def forward(self, x):
        x = torch.relu(self.bn1(self.conv1(x)))  # ReLU after convolution
        x = x.view(-1, 8 * 28 * 28)
        x = torch.sigmoid(self.fc1(x))  # Sigmoid in fully connected layer
        energy = self.fc2(x)
        return energy

# Initialize the EBM, optimizer, and learning rate scheduler
model = ImageEBM()
optimizer = optim.Adam(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)

# Training loop with reduced energy regularization
num_epochs = 5
alpha = 0.1  # Reduced regularization strength
for epoch in range(num_epochs):
    for batch_idx, (data, _) in enumerate(train_loader):
        optimizer.zero_grad()
        energy = model(data)

        # Add energy regularization to the loss
        loss = torch.mean(energy) + alpha * torch.mean(torch.relu(-energy))

        loss.backward()
        optimizer.step()

        if batch_idx % 100 == 0:
            print(
                f"Epoch [{epoch+1}/{num_epochs}], Batch [{batch_idx+1}/{len(train_loader)}], Loss: {loss.item():.4f}"
            )

    scheduler.step()  # Update learning rate

# Evaluate the model
def evaluate(model, test_loader):
    model.eval()  # Set the model to evaluation mode
    test_loss = 0
    with torch.no_grad():
        for data, _ in test_loader:
            energy = model(data)
            test_loss += torch.mean(energy).item()  # Sum up batch loss

    test_loss /= len(test_loader)
    #print(f"Test Loss: {test_loss:.4f}")

evaluate(model, test_loader)

Epoch [1/5], Batch [1/938], Loss: 0.4417
Epoch [1/5], Batch [101/938], Loss: -0.3597
Epoch [1/5], Batch [201/938], Loss: -0.4088
Epoch [1/5], Batch [301/938], Loss: -0.4551
Epoch [1/5], Batch [401/938], Loss: -0.5005
Epoch [1/5], Batch [501/938], Loss: -0.5460
Epoch [1/5], Batch [601/938], Loss: -0.5912
Epoch [1/5], Batch [701/938], Loss: -0.6365
Epoch [1/5], Batch [801/938], Loss: -0.6816
Epoch [1/5], Batch [901/938], Loss: -0.7267
Epoch [2/5], Batch [1/938], Loss: -0.7439
Epoch [2/5], Batch [101/938], Loss: -0.7890
Epoch [2/5], Batch [201/938], Loss: -0.8341
Epoch [2/5], Batch [301/938], Loss: -0.8791
Epoch [2/5], Batch [401/938], Loss: -0.9242
Epoch [2/5], Batch [501/938], Loss: -0.9692
Epoch [2/5], Batch [601/938], Loss: -1.0143
Epoch [2/5], Batch [701/938], Loss: -1.0593
Epoch [2/5], Batch [801/938], Loss: -1.1044
Epoch [2/5], Batch [901/938], Loss: -1.1494
Epoch [3/5], Batch [1/938], Loss: -1.1665
Epoch [3/5], Batch [101/938], Loss: -1.2115
Epoch [3/5], Batch [201/938], Loss: -1.

In [ ]:
# Load MNIST test dataset
test_dataset = datasets.MNIST(
    root="./data", train=False, download=True, transform=transforms.ToTensor()
)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

# Evaluate the model
def evaluate(model, test_loader):
    model.eval()  # Set the model to evaluation mode
    test_loss = 0
    with torch.no_grad():
        for data, _ in test_loader:
            energy = model(data)
            test_loss += torch.mean(energy).item()  # Sum up batch loss

    test_loss /= len(test_loader)
    print(f"Eval Loss: {test_loss:.4f}")

evaluate(model, test_loader)

Eval Loss: -3.1242


## CIFAR-10 Image Classification

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

# Load CIFAR-10 dataset
transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)

train_dataset = datasets.CIFAR10(
    root="./data", train=True, download=True, transform=transform
)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)

test_dataset = datasets.CIFAR10(
    root="./data", train=False, download=True, transform=transform
)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

# Define the ImageEBM for CIFAR-10 (color images)
class ImageEBM(nn.Module):
    def __init__(self):
        super(ImageEBM, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)  # Input channels = 3 for color
        self.bn1 = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(32)
        self.fc1 = nn.Linear(32 * 8 * 8, 128)  # Adjust input size for CIFAR-10
        self.fc2 = nn.Linear(128, 1)

    def forward(self, x):
        x = torch.relu(self.bn1(self.conv1(x)))
        x = torch.max_pool2d(x, 2, 2)
        x = torch.relu(self.bn2(self.conv2(x)))
        x = torch.max_pool2d(x, 2, 2)
        x = x.view(-1, 32 * 8 * 8)  # Adjust size for CIFAR-10
        x = torch.sigmoid(self.fc1(x))
        energy = self.fc2(x)
        return energy

# Initialize the EBM, optimizer, and learning rate scheduler
model = ImageEBM()
optimizer = optim.Adam(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)

# Training loop with energy regularization
num_epochs = 5  # You might need more epochs for CIFAR-10
alpha = 0.1
for epoch in range(num_epochs):
    for batch_idx, (data, _) in enumerate(train_loader):
        optimizer.zero_grad()
        energy = model(data)

        # Add energy regularization to the loss
        loss = torch.mean(energy) + alpha * torch.mean(torch.relu(-energy))

        loss.backward()
        optimizer.step()

        if batch_idx % 100 == 0:
            print(
                f"Epoch [{epoch+1}/{num_epochs}], Batch [{batch_idx+1}/{len(train_loader)}], Loss: {loss.item():.4f}"
            )

    scheduler.step()  # Update learning rate

Files already downloaded and verified
Files already downloaded and verified
Epoch [1/5], Batch [1/782], Loss: -0.3356
Epoch [1/5], Batch [101/782], Loss: -3.6440
Epoch [1/5], Batch [201/782], Loss: -4.3777
Epoch [1/5], Batch [301/782], Loss: -5.0892
Epoch [1/5], Batch [401/782], Loss: -5.8079
Epoch [1/5], Batch [501/782], Loss: -6.5292
Epoch [1/5], Batch [601/782], Loss: -7.2430
Epoch [1/5], Batch [701/782], Loss: -7.9535
Epoch [2/5], Batch [1/782], Loss: -8.5346
Epoch [2/5], Batch [101/782], Loss: -9.2422
Epoch [2/5], Batch [201/782], Loss: -9.9488
Epoch [2/5], Batch [301/782], Loss: -10.6550
Epoch [2/5], Batch [401/782], Loss: -11.3972
Epoch [2/5], Batch [501/782], Loss: -12.1229
Epoch [2/5], Batch [601/782], Loss: -12.8440
Epoch [2/5], Batch [701/782], Loss: -13.5626
Epoch [3/5], Batch [1/782], Loss: -14.1661
Epoch [3/5], Batch [101/782], Loss: -14.9152
Epoch [3/5], Batch [201/782], Loss: -15.6515
Epoch [3/5], Batch [301/782], Loss: -16.4029
Epoch [3/5], Batch [401/782], Loss: -17.1

In [ ]:
import torch
from torchvision import datasets, transforms

# Load CIFAR-10 test dataset
transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)

test_dataset = datasets.CIFAR10(
    root="./data", train=False, download=True, transform=transform
)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)


# Evaluate the model
def evaluate(model, test_loader):
    model.eval()  # Set the model to evaluation mode
    test_loss = 0
    with torch.no_grad():
        for data, _ in test_loader:
            energy = model(data)
            test_loss += torch.mean(energy).item()  # Sum up batch loss

    test_loss /= len(test_loader)
    print(f"Eval Loss: {test_loss:.4f}")

evaluate(model, test_loader)

Files already downloaded and verified
Eval Loss: -34.9541


## CIFAR-10 Image Classification - Avoid Overfitting  



Overfitting is a modeling error that occurs when a model is too closely aligned to a specific set of data, resulting in poor predictions for new data

In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
import torch.nn.functional as F  # Import functional for LeakyReLU

# --- Data Augmentation ---
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(degrees=15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# --- Data Loading ---
train_dataset = datasets.CIFAR10(root="./data", train=True, download=True, transform=transform_train)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)

test_dataset = datasets.CIFAR10(root="./data", train=False, download=True, transform=transform_test)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

# --- Model Definition ---
class ImageEBM(nn.Module):
    def __init__(self):
        super(ImageEBM, self).__init__()
        self.conv1 = nn.Conv2d(3, 4, kernel_size=3, stride=1, padding=1)  # Reduced filters
        self.bn1 = nn.BatchNorm2d(4)
        self.conv2 = nn.Conv2d(4, 8, kernel_size=3, stride=1, padding=1)  # Reduced filters
        self.bn2 = nn.BatchNorm2d(8)
        self.fc1 = nn.Linear(8 * 8 * 8, 16)  # Reduced neurons
        self.fc2 = nn.Linear(16, 1)
        self.dropout = nn.Dropout(p=0.4)  # Increased dropout rate

    def forward(self, x):
        x = F.leaky_relu(self.bn1(self.conv1(x)), negative_slope=0.01)
        x = torch.max_pool2d(x, 2, 2)
        x = F.leaky_relu(self.bn2(self.conv2(x)), negative_slope=0.01)
        x = torch.max_pool2d(x, 2, 2)
        x = x.view(-1, 8 * 8 * 8)
        x = self.dropout(x)
        x = torch.sigmoid(self.fc1(x))
        energy = self.fc2(x)
        return energy

# --- Model Initialization ---
model = ImageEBM()

# --- Optimizer and Scheduler ---
optimizer = optim.Adam(model.parameters(), lr=5e-5, weight_decay=0.01)  # Reduced learning rate, increased weight decay
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=2, factor=0.5)  # More aggressive LR scheduling

# --- Training Loop with Early Stopping ---
def train(model, train_loader, test_loader, optimizer, scheduler, num_epochs, alpha, patience=5):
    best_eval_loss = float('inf')
    epochs_without_improvement = 0

    for epoch in range(num_epochs):
        model.train()
        for batch_idx, (data, _) in enumerate(train_loader):
            optimizer.zero_grad()
            energy = model(data)
            loss = torch.mean(energy) + alpha * torch.mean(torch.relu(-energy))

            loss.backward()
            optimizer.step()

            if batch_idx % 100 == 0:
                print(f"Epoch [{epoch+1}/{num_epochs}], Batch [{batch_idx+1}/{len(train_loader)}], Loss: {loss.item():.4f}")

        # Evaluate the model after each epoch
        model.eval()
        test_loss = 0
        with torch.no_grad():
            for data, _ in test_loader:
                energy = model(data)
                test_loss += torch.mean(energy).item()

        test_loss /= len(test_loader)
        print(f"Eval Loss: {test_loss:.4f}")

        scheduler.step(test_loss)  # Use test loss for LR scheduling

        # Early stopping check
        if test_loss < best_eval_loss:
            best_eval_loss = test_loss
            epochs_without_improvement = 0
        else:
            epochs_without_improvement += 1
            if epochs_without_improvement >= patience:
                print(f"Early stopping at epoch {epoch+1}")
                break

# --- Train the Model ---
num_epochs = 50
alpha = 1000  # Significantly increased alpha
train(model, train_loader, test_loader, optimizer, scheduler, num_epochs, alpha)


# Evaluate the model
def evaluate(model, test_loader):
    model.eval()  # Set the model to evaluation mode
    test_loss = 0
    with torch.no_grad():
        for data, _ in test_loader:
            energy = model(data)
            test_loss += torch.mean(energy).item()  # Sum up batch loss

    test_loss /= len(test_loader)
    print(f"Eval Loss: {test_loss:.4f}")

evaluate(model, test_loader)

Files already downloaded and verified
Files already downloaded and verified
Epoch [1/50], Batch [1/782], Loss: 14.9660
Epoch [1/50], Batch [101/782], Loss: 0.2232
Epoch [1/50], Batch [201/782], Loss: 0.2384
Epoch [1/50], Batch [301/782], Loss: 0.5560
Epoch [1/50], Batch [401/782], Loss: 0.2187
Epoch [1/50], Batch [501/782], Loss: 0.2130
Epoch [1/50], Batch [601/782], Loss: 0.2062
Epoch [1/50], Batch [701/782], Loss: 0.2179
Eval Loss: 0.1531
Epoch [2/50], Batch [1/782], Loss: 0.2182
Epoch [2/50], Batch [101/782], Loss: 0.2050
Epoch [2/50], Batch [201/782], Loss: 0.1982
Epoch [2/50], Batch [301/782], Loss: 0.2004
Epoch [2/50], Batch [401/782], Loss: 0.2096
Epoch [2/50], Batch [501/782], Loss: 0.3405
Epoch [2/50], Batch [601/782], Loss: 0.1677
Epoch [2/50], Batch [701/782], Loss: 0.2142
Eval Loss: 0.1458
Epoch [3/50], Batch [1/782], Loss: 0.1773
Epoch [3/50], Batch [101/782], Loss: 0.1832
Epoch [3/50], Batch [201/782], Loss: 0.1833
Epoch [3/50], Batch [301/782], Loss: 0.1685
Epoch [3/50],